# Heim & Kratzer Chapter 3: Semantic Rules with Backtick DSL

This notebook demonstrates the Heim & Kratzer (1998) Chapter 3 semantic rules using a toy lexicon, the `Interpreter` class, and the ultra-light backtick DSL for `PhiValue` literals.

## Import Required Libraries

Import all necessary modules, including `Interpreter`, `Tree`, `PhiValue`, and the backtick DSL installer.

In [1]:
from phosphorus.semantics.interpret import Interpreter
from phosphorus.syntax.tree import Tree
from phosphorus.core.phivalue import PhiValue
from phosphorus.core.stypes import takes
from phosphorus.core.constants import UNDEF

# Install the backtick DSL for PhiValue literals
import phosphorus.dsl.backtick


             _    _                  _    _
            | |  | |                | |  | |
           _| |_ | |__   ___  ___  _| |_ | |__   ___  _ __ _   _  ____
          /     \| '_ \ / _ \/ __|/     \| '_ \ / _ \| '__| | | |/ ___)
         ( (| |) ) | | | (_) \__ ( (| |) ) | | | (_) | |  | |_| ( (__
          \_   _/|_| |_|\___/|___/\_   _/|_| |_|\___/|_|   \__,_|\__ \
            | |                     | |                            _) )
            |_|                     |_|                           (__/

        Welcome to the Phosphorus Meaning Engine v3
        Created by Ezra Keshet (EzraKeshet.com)




## Define Lexicon

Create a dictionary mapping lexical items (e.g., 'john', 'mary', 'runs', 'loves') to their `PhiValue` representations using the backtick DSL.

In [2]:
lexicon = {
  "john":   `JOHN.e,
  "mary":   `MARY.e,
  "runs":   `lambda x=e: RUN(x).t,
  "loves":  `lambda y=e: (lambda x=e: LOVE(x,y).t),
}

## Create Interpreter and Register Rules

Instantiate an `Interpreter` named `interpret3`, assign the lexicon, and define the TN, NN, and FA rules directly in the notebook.

In [3]:
interpret3 = Interpreter(lexicon=lexicon)

# Terminal Node (TN): lexical lookup
@interpret3.rule()
def TN(*, alpha: str):    
  """Terminal Node: lexical lookup of *alpha* (string token)."""
  return interpret3.lookup(alpha)

# Non-Branching Node (NN): pass child meaning unchanged
@interpret3.rule()
def NN(beta: PhiValue):
  """Non-branching Node: pass child meaning unchanged."""
  return beta

# Functional Application (FA): apply function to argument
@interpret3.rule()
def FA(beta: PhiValue, gamma: PhiValue):
  """Functional Application (order determined by `takes`)."""
  if takes(beta, gamma):
      fn, arg = beta, gamma
  elif takes(gamma, beta):
      fn, arg = gamma, beta
  else:
      return UNDEF

  return `fn(arg)

## Test Example Sentences

Parse and interpret example syntactic trees using `interpret3`, displaying both the tree and its semantic interpretation.

In [4]:
examples = [
  "(N John)",
  "(DP (N John))",
  "(S (N John) (V runs))",
  "(S (N John) (VP (V loves) (N Mary)))",
]

In [8]:
from IPython.display import display, HTML

for src in examples:
  tree = Tree.fromstring(src)
  meaning = interpret3.interpret(tree)
  tree_html = tree._repr_svg_()  # SVG as HTML string
  meaning_html = meaning._repr_html_()  # HTML as string
  display(HTML(f"""
    <table>
      <tr>
        <td style="vertical-align:top;min-width:18ch;text-align:left">{tree_html}</td>
        <td style="vertical-align:top;padding-left:2em">{meaning_html}</td>
      </tr>
    </table>
  """))

NJohn,JOHN e


DPNJohn,JOHN e


SNJohnVruns,RUN(JOHN) t


SNJohnVPVlovesNMary,"LOVE(JOHN, MARY) t"
